In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("/kaggle/input/sf-crime/train.csv.zip")
test=pd.read_csv("/kaggle/input/sf-crime/test.csv.zip")
pd.options.display.max_columns=99
display(train,test)

train,test 각 데이터에서 겹치지 않는 값과 정답 값을 drop하여 정제합니다

In [ ]:
alldata=pd.concat([train,test],axis=0)
alldata2=alldata.drop(columns=["Category","Descript","Resolution","Dates"])
alldata2

날짜를 숫자로 변경하여 러닝에 용이합니다

In [ ]:
alldata2["DayOfWeek"].unique()
alldata2["DayOfWeek"]=alldata2["DayOfWeek"].replace({"Wednesday":0, "Tuesday":1, "Monday":2, "Sunday":3, "Saturday":4, "Friday":5, "Thursday":6})
alldata2["DayOfWeek"]


LabelEncoder를 이용하면 해당 칼럼의 값을 임의의 숫자로 지정해줍니다

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
alldata2["PdDistrict"]=le.fit_transform(alldata2["PdDistrict"])
alldata2

PdDistrict의 각 칼럼 고유값에 해당하는 숫자 값을 dictionary로 만들어 줍니다

In [ ]:
len(le.classes_) #10개
dict(zip(le.classes_,range(10)))


칼럼 값이 object인 것이 있는지 추가로 확인합니다.

In [ ]:
category=alldata2.columns[alldata2.dtypes == object]
category

Address column에서 레이블 인코딩 처리, 숫자로 변경해 줍니다

In [ ]:
for col in category:
    alldata2[col]=le.fit_transform(alldata2["Address"])

In [ ]:
alldata2

In [ ]:
alldata2=alldata2.fillna(-1) # nan값 있을 경우 러닝 과정에서 오류발생
alldata2

In [ ]:
train2=alldata2[:len(train)]
test2=alldata2[len(train):]

RandomForestClassifier 에서 score 9.53751
GBDT와 달리 병렬로 트리를 작성. 각 결정 트리의 학습에서 행 데이터나 특징을 샘플링해 전달함으로써 다양한 결정 트리를 작성하고, 앙상블하여 일반화 성능이 높은 예측을 실시. 모델 구축 순서
1) 학습 데이터에서 행 데이터를 샘플링하여 추출
2) 1)을 학습하고 결정트리를 작성, 분기를 작성할 때는 특징의 일부만을 샘플링하여 추출하고 특징의 후보로 삼음. 그 후보들로 부터 데이터를 가장 잘 분할하는 특징과 임곗값을 선택해 분기로 삼음
3) 1)과 2)의 과정을 결정트리의 개수만큼 병렬로 수행

-회귀문제일때는 제곱오차, 분류문제일때는 지니 불순도가 가장 감소하도록 분기를 시행
-결정트리마다 원래 개수와 같은 수만큼 행 데이터를 복원 추출하는 부트스트랩 샘플링을 실시.
부트스트랩 샘플링에서는 중복 추출되는 행 데이터가 있는 한편, 평균적으로 행데이터의 1/3 정도는 추출되지 않음
-분기마다 특징의 일부를 샘플링한 것을 후보로 삼고 그중 분기의 특징을 선택. 회귀 문제에서는 샘플링하지 않고 모든 특징을 후보로 삼음. 분류문제에서는 특징개수의 제곱근 개수만큼 추출하여 후보로 삼음

*결정 트리의 개수와 모델 성능의 관계
결정 트리를 병렬로 작성하므로, GBDT와는 달리 결정 트리의 개수가 지나치게 증가하여 모델 성능이 낮아지는 일은 없음.
어느 정도 증가한 후에는 성능이 더 이상 올라가지 않음. 결정 트리의 개수는 계산 시간과 성능의 트레이드 오프로 결정

*예측 확률 타당성
> 분류 작업시 GBDT에서는 가중치에 기반을 둔 예측 확률의 로그 손실을 최소화하려는 반면, 랜덤 포레스트에서는 지니 불순도를 최소화하려는 각 결정 트리의 예측값의 평균을 구함.

*랜덤 포레스트 방법의 예측 확률 타당성 보증이 되지 않음
1) 데이터가 충분하지 않을 경우, 0이나 1에 가까운 확률을 예측하기 어려움
2) 모델이 로그 손실을 최소화 하도록 학습할 경우, 충분한 데이터가 있으면 타당한 확률을 예측하나 그렇지 않으면 왜곡
> GBDT,신경망, 로지스틱 회귀일때 분류문제에서는 통상적인 설정으로 로그 손실을 목적함수로 삼아 학습하지만 랜덤포레스트에서는 다른 알고리즘으로 분류하므로 확률이 왜곡됨


In [ ]:
# from sklearn.ensemble import RandomForestClassifier 
# rfc=RandomForestClassifier(n_jobs=-1)
# rfc.fit(train2,train["Category"])


In [ ]:
# result=rfc.predict_proba(test2)
# result

CatBoostClassifier에서 score 2.41086으로 상승 (score 낮을 수록 순위 상승)
GBDT의 라이브러리인 CATBOOST는 범주형 변수 지원, 과적합 해소를 위한 정렬된 부스팅, GPU를 활용한 트리모델에서의 부스팅 성능 향상과 같은 기능을 지원
범주형 변수로서 지정한 특징을 자동으로 타깃 인코딩하여 수치로 변환. 타깃 인코딩을 잘못 사용하면 목적변수의 정보를 부적절하게 사용. 랜덤하게 데이터를 정렬하면서 적용하는 식의 연구 진행. 결정 트리를 작성하는 과정에서 동적으로 범주형 변수의 조합에 대해 타깃 인코딩이 이루어짐. 즉 어떤 분기에서 범주형 변수가 사용되었을 때 그 범주형 변수와 다른 범주형 변수와의 조합에 대해 타깃 인코딩 연산이 이루어지며 급다 깊은 분기에서 그 결과가 사용

데이터 수가 적을 때는 정렬부스팅이라는 알고리즘 사용. 속도는 느리지만 데이터 수가 적을 때 모델 성능이 높음.

In [ ]:
from catboost import CatBoostClassifier
cbc=CatBoostClassifier (task_type="GPU")
cbc.fit(train2,train["Category"])

result=cbc.predict_proba(test2)
result

**GBM에 비해 시간은 적게 걸리고, RANDOM FOREST에 비해 성능이 좋다.
약한 학습기를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식이기 때문에**

GBM(Gradient Boosting Machine)
부스팅 알고리즘은 여러개의 약한 학습기를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식. 랜덤 포레스트보다는 예측성능이 조금 뛰어난 경우가 많음. 그러나 수행시간이 오래 걸리고, 하이퍼 파라미터 튜닝 노력도 더 필요. 약한 학습기의 순차적인 예측 오류 보정을 통해 학습을 수행하므로 멀티 cpu코어 시스템을 사용하더라도 병렬 처리가 되지않아서 대용량 데이터의 경우 학습에 매우 많은 시간이 필요. => 5시간 이상 소요되어 pause

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
# gbc=GradientBoostingClassifier(random_state=0)
# gbc.fit(train2,train["Category"])
# result=gbc.predict_proba(test2)
# result

In [ ]:
sub=pd.read_csv("/kaggle/input/sf-crime/sampleSubmission.csv.zip")
sub

In [ ]:
sub.iloc[:,1:]=result
sub

In [ ]:
RandomForestClassifier를 사용하면 score가 9.53751 이었으나,CatBoostClassifier를 사용하면 score 2.41086으로 개선되었습니다. (score 낮을 수록 순위가 상승합니다)
RandomForestClassifier는 로그 손실을 최소화 하는 방향이나 과정에서 예측 오류에 대한 가중치를 적용하지 않습니다.
CatBoostClassifier는 예측 과정에서 가중치를 적용하여 오류를 보정하기 때문에 시간은 많이 들지만 더 정확한 예측이 가능합니다.

In [ ]:
sub.to_csv("cbc.csv",index=False)